In [1]:
from datetime import date, datetime

from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, select, event, Date, DateTime, exists, and_, UniqueConstraint, ForeignKeyConstraint
from sqlalchemy.orm import Session, declarative_base, relationship

In [2]:
# Creating the engine

engine = create_engine("sqlite:///:memory:", echo=True, future=True)

In [3]:
# Adding foreign key pragma on every connection for sqlite by event

def _fk_pragma_on_connect(dbapi_con, con_record):
    dbapi_con.execute('PRAGMA foreign_keys = 1')

if engine.url.drivername == 'sqlite':
    event.listen(engine, 'connect', _fk_pragma_on_connect)
    print("Added FOREIGN_KEY pragma event for sqlite.")

Added FOREIGN_KEY pragma event for sqlite.


In [4]:
# Creating the base class for table-mapped data-classes

Base = declarative_base()

In [5]:
# Declaration of table-mapped data-classes

class User(Base):
    __tablename__ = 'user_accounts'
    __table_args__ = (UniqueConstraint('name', 'fullname'),)

    # id = Column(Integer)
    name = Column(String(30), primary_key=True)
    fullname = Column(String, primary_key=True)
    date_column = Column(Date)
    datetime_column = Column(DateTime)

    addresses = relationship("Address", back_populates="user")
    
    # def __repr__(self):
    #    return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r}, date_column={self.date_column!r}, datetime_column={self.datetime_column})"

    def __eq__(self, other):
        return (
            (self.name == other.name) and
            (self.fullname == other.fullname))


class Address(Base):
    __tablename__ = 'addresses'
    __table_args__ = (ForeignKeyConstraint(["user_name", "user_fullname"], ["user_accounts.name", "user_accounts.fullname"]),)

    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_name = Column(String(30))
    user_fullname = Column(String)

    user = relationship(
        "User", 
        foreign_keys="[Address.user_name, Address.user_fullname]",
        back_populates="addresses")

    # def __repr__(self):
    #     return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [6]:
# Create tables for all data-classes

Base.metadata.create_all(engine)

# Erstellt nur notwendige Elemente in der DB. Bereits vorhandene Elemente (Tabellen etc. ) werden ohne Fehler ignoriert (checkfirst=True).

2022-03-06 23:32:00,759 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-06 23:32:00,760 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_accounts")
2022-03-06 23:32:00,760 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-06 23:32:00,761 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_accounts")
2022-03-06 23:32:00,762 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-06 23:32:00,763 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2022-03-06 23:32:00,763 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-06 23:32:00,764 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("addresses")
2022-03-06 23:32:00,765 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-06 23:32:00,766 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_accounts (
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR NOT NULL, 
	date_column DATE, 
	datetime_column DATETIME, 
	PRIMARY KEY (name, fullname), 
	UNIQUE (name, fullname)
)


2022-03-06 23:32:00,767 INFO sqlalchemy.en

In [7]:
# Create data-objects

sandy = User(name="sandy", fullname="Sandy Cheeks", date_column=date.today(), datetime_column=datetime.now())
sandy2 = User(name="sandy", fullname="Sandy Cheeks", date_column=date.today(), datetime_column=datetime.now())
sandy3 = User(name="sandy", fullname="Sandy Cheeks")
sandy4 = User(name="sandy", fullname="Sandy Cheeks")

In [8]:
sandy3 is sandy4

False

In [9]:
sandies = [sandy, sandy2]
sandy4 in sandies

True

In [10]:
# Adding users to the database

session = Session(engine)
session.add(sandy)
# session.add_all([sandy, sandy2])
# session.commit()


In [11]:
sandy in session

True

In [12]:
with Session(engine) as session:
    print(session.execute(select(User)).all())

2022-03-06 23:32:01,286 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-06 23:32:01,289 INFO sqlalchemy.engine.Engine SELECT user_accounts.name, user_accounts.fullname, user_accounts.date_column, user_accounts.datetime_column 
FROM user_accounts
2022-03-06 23:32:01,289 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
[]
2022-03-06 23:32:01,290 INFO sqlalchemy.engine.Engine ROLLBACK
